# Régression pour retrouver le temps de livraison

Ruben SAILLY, Frédéric EGENSCHEVILLER


## Installation des packages nécessaires

In [1]:
!pip install pandas numpy scikit-learn matplotlib seaborn kagglehub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 58.0 MB/s  0:00:00m0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 84.3 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 69.1 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 79.5 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 53.2 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.5/35.5 MB 77.1 MB/s  0:00:006m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15/15 [seaborn]4/15 [seaborn]ib]n]


## Import des données et visualisation

On va télécharger le dataset via kagglehub, puis on va visualiser les données, afin de voir ce que l'on va devoir faire pour les nettoyer.
Le nettoyage se fera dans un second temps avec une pipeline scikit-learn dédiée.

In [4]:
import kagglehub
import pandas as pd
path = kagglehub.dataset_download("gauravmalik26/food-delivery-dataset")
print("Path to dataset files:", path)

df_test = pd.read_csv(path + "/test.csv")
df_train = pd.read_csv(path + "/train.csv")

Path to dataset files: /root/.cache/kagglehub/datasets/gauravmalik26/food-delivery-dataset/versions/1


Les fichiers téléchargés sont les suivants :
- ```test.csv```: le dataset de test, avec les features mais sans la target (temps de livraison)
- ```train.csv```: le dataset d'entrainement, avec les features et la target (temps de livraison)
- ```sample_submission.csv```: un exemple de fichier de soumission pour kaggle, avec ids de ```test.csv``` et des valeurs quelconques pour la target.

On va donc utiliser ```train.csv```, étant donné que c'est le seul qui contient la target, et que ```test.csv``` sert à évaluer notre modèle sur kaggle.

Regardons les données d'entrainement.

In [14]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45593 entries, 0 to 45592
Data columns (total 20 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   ID                           45593 non-null  object 
 1   Delivery_person_ID           45593 non-null  object 
 2   Delivery_person_Age          45593 non-null  object 
 3   Delivery_person_Ratings      45593 non-null  object 
 4   Restaurant_latitude          45593 non-null  float64
 5   Restaurant_longitude         45593 non-null  float64
 6   Delivery_location_latitude   45593 non-null  float64
 7   Delivery_location_longitude  45593 non-null  float64
 8   Order_Date                   45593 non-null  object 
 9   Time_Orderd                  45593 non-null  object 
 10  Time_Order_picked            45593 non-null  object 
 11  Weatherconditions            45593 non-null  object 
 12  Road_traffic_density         45593 non-null  object 
 13  Vehicle_conditio

On voit que les données sont majoritairement des chaînes de caractères (object), même la target. Regardons les premières lignes du dataset pour toutes les colonnes, afin de distinguer :
- ce qui est numérique
- ce qui est catégoriel
- ce qui est textuel

In [12]:
df_train.head()

,ID,Delivery_person_ID,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Order_Date,Time_Orderd,Time_Order_picked,Weatherconditions,Road_traffic_density,Vehicle_condition,Type_of_order,Type_of_vehicle,multiple_deliveries,Festival,City,Time_taken(min)
0,0x4607,INDORES13DEL02,37,4.9,22.745049,75.892471,22.765049,75.912471,19-03-2022,11:30:00,11:45:00,conditions Sunny,High,2,Snack,motorcycle,0,No,Urban,(min) 24
1,0xb379,BANGRES18DEL02,34,4.5,12.913041,77.683237,13.043041,77.813237,25-03-2022,19:45:00,19:50:00,conditions Stormy,Jam,2,Snack,scooter,1,No,Metropolitian,(min) 33
2,0x5d6d,BANGRES19DEL01,23,4.4,12.914264,77.678400,12.924264,77.688400,19-03-2022,08:30:00,08:45:00,conditions Sandstorms,Low,0,Drinks,motorcycle,1,No,Urban,(min) 26
3,0x7a6a,COIMBRES13DEL02,38,4.7,11.003669,76.976494,11.053669,77.026494,05-04-2022,18:00:00,18:10:00,conditions Sunny,Medium,0,Buffet,motorcycle,1,No,Metropolitian,(min) 21
4,0x70a2,CHENRES12DEL01,32,4.6,12.972793,80.249982,13.012793,80.289982,26-03-2022,13:30:00,13:45:00,conditions Cloudy,High,1,Snack,scooter,1,No,Metropolitian,(min) 30


In [20]:
df_train.nunique()

ID                             45593
Delivery_person_ID              1320
Delivery_person_Age               23
Delivery_person_Ratings           29
Restaurant_latitude              657
Restaurant_longitude             518
Delivery_location_latitude      4373
Delivery_location_longitude     4373
Order_Date                        44
Time_Orderd                      177
Time_Order_picked                193
Weatherconditions                  7
Road_traffic_density               5
Vehicle_condition                  4
Type_of_order                      4
Type_of_vehicle                    4
multiple_deliveries                5
Festival                           3
City                               4
Time_taken(min)                   45
dtype: int64

On peut déjà affirmer les choses suivantes :
- ```ID``` est une chaine de caractères, et est un nombre hexadécimal. Il est unique pour chaque ligne, donc il ne servira pas pour la régression, on le supprimera.
- ```Delivery_person_ID``` est aussi une chaine de caractères, mais elle n'est pas unique, donc elle peut servir pour la régression. C'est une variable catégorielle.
- ```Delivery_person_Age``` est numérique, mais est stockée en tant que chaîne de caractères. Il faudra la convertir en numérique.
- ```Delivery_person_Ratings``` est numérique, mais est stockée en tant que chaîne de caractères. Il faudra la convertir en numérique.
- ```Restaurant_latitude```, ```Restaurant_longitude```, ```Delivery_location_latitude```, ```Delivery_location_longitude``` sont des coordonnées GPS, stockées en numérique. On pourra en déduire la distance entre le restaurant et le lieu de livraison, qui sera une feature intéressante.
- ```Order_Date``` est une date, stockée en tant que chaîne de caractères. On pourra en extraire des informations intéressantes, comme le jour de la semaine, le mois, etc.
- ```Time_Orderd``` et ```Time_Order_picked``` sont des heures, stockées en tant que chaînes de caractères. On pourra en extraire des informations intéressantes, comme l'heure de la journée. Étant donné que l'on a pas de valeurs manquantes dans ces colonnes, et que il y a dans chaque colonne 200 valeurs distinctes, on peut les traiter comme des variables catégorielles.
- ```Weather_conditions```, ```Road_traffic_density```, ```Type_of_order```, ```Type_of_vehicle```, ```Festival```, ```City``` sont des variables catégorielles, stockées en tant que chaînes de caractères.
- ```multiple_deliveries``` est numérique, mais est stockée en tant que chaîne de caractères. Il faudra la convertir en numérique.
- ```Time_taken (min)``` est la target, elle est numérique mais stockée en tant que chaîne de caractères. Il faudra la convertir en numérique. On va devoir faire une fonction de nettoyage pour nettoyer les ```(min)```présents dans les valeurs.